In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

import librosa
import librosa.display

import torch
import torch.nn as nn
from torchsummary import summary

from common_audio import *
from audio_dataset import *

torch.manual_seed(0)
np.random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

print(device)


cpu


In [24]:
def cool_new_loss(vh, v, n): #not vectorized yet
    M = torch.stack((v, n),axis=0).transpose(0, 1)
    ans = torch.inverse(M.transpose(0, 1)@M)@M.transpose(0, 1) @ vh
    e = torch.norm(vh-ans[0]*v - ans[1]*n) # want this to be 0
    
    loss = 0*(ans[0] - 1.)**2 + 1*(ans[1] - 0.)**2 #+ (e**2)
    
    return loss
    
def mse_loss(vh, v):
    return ((vh-v)**2).mean()

def test(vh, v, n):
    M = torch.stack((v, n),axis=0).transpose(0, 1)
    ans = torch.inverse(M.transpose(0, 1)@M)@M.transpose(0, 1) @ vh
    e = torch.norm(vh-ans[0]*v - ans[1]*n) # want this to be 0
    return ans[0].item(), ans[1].item(), e.item()

In [25]:
torch.manual_seed(0)

z_dim = 1000

v = 1000*torch.randn(z_dim)
n = 1000*torch.randn(z_dim)

m = v+n

vh = m.clone().requires_grad_()
opt = torch.optim.Adam([vh], lr=1)

print(mse_loss(vh, v).detach())
print(test(vh, v, n))

for steps in range(1000):
    opt.zero_grad()
    loss = mse_loss(vh, v)
    loss.backward()
    opt.step()
    
print(mse_loss(vh, v).detach())
print(test(vh, v, n))

tensor(943321.2500)
(1.0, 1.000000238418579, 0.007320504169911146)
tensor(195350.0625)
(1.0016117095947266, 0.4160824120044708, 5662.10986328125)


In [26]:
torch.manual_seed(0)

z_dim = 1000

v = 1000*torch.randn(z_dim)
n = 1000*torch.randn(z_dim)

m = v+n

vh = m.clone().requires_grad_()
opt = torch.optim.Adam([vh], lr=1)

print(mse_loss(vh, v).detach())
print(test(vh, v, n))

for steps in range(1000):
    opt.zero_grad()
    loss = cool_new_loss(vh, v, n)
    loss.backward()
    opt.step()
    
print(mse_loss(vh, v).detach())
print(test(vh, v, n))

tensor(943321.2500)
(1.0, 1.000000238418579, 0.007320504169911146)
tensor(331935.9688)
(0.9891633987426758, 0.35580939054489136, 14568.7197265625)


In [23]:
X = torch.tensor([1.,0,0])
Y = torch.tensor([0.,1,0])
p = X+Y + torch.tensor([0,0,0])
p.requires_grad_()

print(test(p, X, Y))

opt = torch.optim.Adam([p], lr=0.01)
for steps in range(10):
    opt.zero_grad()
    loss = mse_loss(p, X)
    loss.backward()
    opt.step()
    
print(test(p, X, Y))

(1.0, 1.0, 0.0)
(1.0, 0.9003496766090393, 0.0)
